In [2]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("windows_log.csv")
df

,timestamp,event_id,user,ip,logon_type
0,2024-01-01 09:00:01,4624,user1,10.0.0.1,3
1,2024-01-01 09:01:10,4625,user1,10.0.0.1,3
2,2024-01-01 09:01:20,4625,user1,10.0.0.1,3
3,2024-01-01 09:02:00,4624,admin,10.0.0.2,10
4,2024-01-01 09:05:30,4769,svc_sql,10.0.0.3,3
5,2024-01-01 09:05:40,4769,svc_sql,10.0.0.3,3
6,2024-01-01 09:06:10,4625,user2,10.0.0.9,3
7,2024-01-01 09:06:12,4625,user2,10.0.0.9,3
8,2024-01-01 09:06:14,4625,user2,10.0.0.9,3
9,2024-01-01 09:06:16,4625,user2,10.0.0.9,3


In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,timestamp,event_id,user,ip,logon_type
0,2024-01-01 09:00:01,4624,user1,10.0.0.1,3
1,2024-01-01 09:01:10,4625,user1,10.0.0.1,3
2,2024-01-01 09:01:20,4625,user1,10.0.0.1,3
3,2024-01-01 09:02:00,4624,admin,10.0.0.2,10
4,2024-01-01 09:05:30,4769,svc_sql,10.0.0.3,3
5,2024-01-01 09:05:40,4769,svc_sql,10.0.0.3,3
6,2024-01-01 09:06:10,4625,user2,10.0.0.9,3
7,2024-01-01 09:06:12,4625,user2,10.0.0.9,3
8,2024-01-01 09:06:14,4625,user2,10.0.0.9,3
9,2024-01-01 09:06:16,4625,user2,10.0.0.9,3


In [5]:
df["event_id"].value_counts()

event_id
4625    9
4624    5
4769    2
Name: count, dtype: int64

In [6]:
failed = df[df["event_id"] == 4625]
failed

,timestamp,event_id,user,ip,logon_type
1,2024-01-01 09:01:10,4625,user1,10.0.0.1,3
2,2024-01-01 09:01:20,4625,user1,10.0.0.1,3
6,2024-01-01 09:06:10,4625,user2,10.0.0.9,3
7,2024-01-01 09:06:12,4625,user2,10.0.0.9,3
8,2024-01-01 09:06:14,4625,user2,10.0.0.9,3
9,2024-01-01 09:06:16,4625,user2,10.0.0.9,3
11,2024-01-01 09:10:00,4625,admin,10.0.0.9,10
12,2024-01-01 09:10:03,4625,admin,10.0.0.9,10
14,2024-01-01 09:13:00,4625,svc_sql,10.0.0.3,3


In [14]:
failed["user"].value_counts()

user
user2      4
user1      2
admin      2
svc_sql    1
Name: count, dtype: int64

In [15]:
failed["ip"].value_counts()

ip
10.0.0.9    6
10.0.0.1    2
10.0.0.3    1
Name: count, dtype: int64

In [17]:
failed.groupby(["user", "ip"]).size().sort_values(ascending=False)

user     ip      
user2    10.0.0.9    4
admin    10.0.0.9    2
user1    10.0.0.1    2
svc_sql  10.0.0.3    1
dtype: int64

短期間連続→brute forceの可能性を確認

In [18]:
failed_sorted = failed.sort_values("timestamp")
failed_sorted[["timestamp", "user", "ip", "logon_type"]]

,timestamp,user,ip,logon_type
1,2024-01-01 09:01:10,user1,10.0.0.1,3
2,2024-01-01 09:01:20,user1,10.0.0.1,3
6,2024-01-01 09:06:10,user2,10.0.0.9,3
7,2024-01-01 09:06:12,user2,10.0.0.9,3
8,2024-01-01 09:06:14,user2,10.0.0.9,3
9,2024-01-01 09:06:16,user2,10.0.0.9,3
11,2024-01-01 09:10:00,admin,10.0.0.9,10
12,2024-01-01 09:10:03,admin,10.0.0.9,10
14,2024-01-01 09:13:00,svc_sql,10.0.0.3,3


In [19]:
failed_sorted["user_ip"] = failed_sorted["user"] + "@" + failed_sorted["ip"]
failed_sorted["prev_time"] = failed_sorted.groupby("user_ip")["timestamp"].shift(1)
failed_sorted["delta_sec"] = (failed_sorted["timestamp"] - failed_sorted["prev_time"]).dt.total_seconds()

failed_sorted[["timestamp","user","ip","delta_sec"]]

,timestamp,user,ip,delta_sec
1,2024-01-01 09:01:10,user1,10.0.0.1,NaN
2,2024-01-01 09:01:20,user1,10.0.0.1,10.0
6,2024-01-01 09:06:10,user2,10.0.0.9,NaN
7,2024-01-01 09:06:12,user2,10.0.0.9,2.0
8,2024-01-01 09:06:14,user2,10.0.0.9,2.0
9,2024-01-01 09:06:16,user2,10.0.0.9,2.0
11,2024-01-01 09:10:00,admin,10.0.0.9,NaN
12,2024-01-01 09:10:03,admin,10.0.0.9,3.0
14,2024-01-01 09:13:00,svc_sql,10.0.0.3,NaN


In [22]:
bursty = failed_sorted[failed_sorted["delta_sec"] <= 5]
bursty[["timestamp","user","ip","delta_sec"]]

,timestamp,user,ip,delta_sec
7,2024-01-01 09:06:12,user2,10.0.0.9,2.0
8,2024-01-01 09:06:14,user2,10.0.0.9,2.0
9,2024-01-01 09:06:16,user2,10.0.0.9,2.0
12,2024-01-01 09:10:03,admin,10.0.0.9,3.0
